In [3]:
import pandas as pd
from colorama import Fore, Style


def color_text(predicted, actual):
    colored_text = []
    for team in predicted:
        if team in actual:
            colored_text.append(Fore.GREEN + team + Style.RESET_ALL)
        else:
            colored_text.append(Fore.RED + team + Style.RESET_ALL)

    return ', '.join(colored_text)


def calculate_playoff_accuracy(year, sorted_team_list, data_file):

    results_df = pd.read_csv(data_file)

    year_df_we = results_df[(results_df['year'] == year)
                            & (results_df['confID'] == 'WE')]
    year_df_ea = results_df[(results_df['year'] == year)
                            & (results_df['confID'] == 'EA')]

    actual_we_playoffs = year_df_we[year_df_we['playoff']
                                    == 'Y']['tmID'].tolist()
    actual_ea_playoffs = year_df_ea[year_df_ea['playoff']
                                    == 'Y']['tmID'].tolist()

    all_we_teams = year_df_we['tmID'].tolist()
    all_ea_teams = year_df_ea['tmID'].tolist()

    team_conf_dict = {}

    for team in all_we_teams:
        team_conf_dict[team] = 'WE'
    for team in all_ea_teams:
        team_conf_dict[team] = 'EA'

    predicted_we_playoffs = []
    predicted_ea_playoffs = []

    for team in sorted_team_list:
        try:
            if team_conf_dict[team] == 'WE':
                predicted_we_playoffs.append(team)
            elif team_conf_dict[team] == 'EA':
                predicted_ea_playoffs.append(team)
        except:
            print(
                f"Error: team '{team}' is not even present for the year {year}!")
            return

    we_incorrect = []
    ea_incorrect = []
    we_correct = []
    ea_correct = []

    for team in sorted_team_list:
        if team_conf_dict[team] == 'WE':
            if team in actual_we_playoffs:
                we_correct.append(team)
            else:
                we_incorrect.append(team)
        else:
            if team in actual_ea_playoffs:
                ea_correct.append(team)
            else:
                ea_incorrect.append(team)

    we_correct_count = len(we_correct)
    ea_correct_count = len(ea_correct)

    total_accuracy = ((we_correct_count + ea_correct_count) / 8.0) * 100

    print("=" * 40)
    print(f"{'Year:':<20}{year}")
    print("=" * 40)
    print("WE")
    print(f"{'Guesses:':<15}{color_text(we_correct, actual_we_playoffs) + ', ' + color_text(we_incorrect, actual_we_playoffs) }")
    print(f"{'Missed:':<15}{', '.join(set(actual_we_playoffs) - set(we_correct))}")
    print("=" * 40)
    print("EA")
    print(f"{'Guesses:':<15}{color_text(ea_correct, actual_ea_playoffs) + ', ' + color_text(ea_incorrect, actual_ea_playoffs)}")
    print(f"{'Missed:':<15}{', '.join(set(actual_ea_playoffs) - set(ea_correct))}")
    print("=" * 40)
    print(f"{'Total accuracy:':<20}{total_accuracy:.2f}%")


# Example
year_to_check = 7
team_guesses = ['DET', 'SAS', 'HOU', 'WAS', 'CHA', 'IND', 'SAC', 'PHO']
data_file = "dataset/results.csv"
calculate_playoff_accuracy(year_to_check, team_guesses, data_file)


Year:               7
WE
Guesses:       HOU, SAC, SAS, PHO
Missed:        LAS, SEA
EA
Guesses:       DET, WAS, IND, CHA
Missed:        CON
Total accuracy:     62.50%


In [2]:
team_guesses = ['DET', 'SAS', 'HOU', 'WAS', 'CHA', 'IND', 'SAC', 'PHO']
            #   EA      WE     WE      EA     EA     EA     WE     WE
